In [1]:
import sys,os
if not(os.path.isfile("train_nominal_000.h5")):
    if sys.platform=="darwin": #MAC OSX
        !curl https://opendata.cern.ch/record/80030/files/assets/atlas/datascience/CERN-EP-2024-159/train_nominal_000.h5.gz --output train_nominal_000.h5.gz
        !curl https://opendata.cern.ch/record/80030/files/assets/atlas/datascience/CERN-EP-2024-159/test_nominal_000.h5.gz --output test_nominal_000.h5.gz
    elif sys.platform == "linux" or sys.platform == "linux2":
        !wget https://opendata.cern.ch/record/80030/files/assets/atlas/datascience/CERN-EP-2024-159/train_nominal_000.h5.gz
        !wget https://opendata.cern.ch/record/80030/files/assets/atlas/datascience/CERN-EP-2024-159/test_nominal_000.h5.gz
    !gunzip -f train_nominal_000.h5.gz
    !gunzip -f test_nominal_000.h5.gz

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader, ConcatDataset
from torchinfo import summary

from sklearn import metrics
import matplotlib.pyplot as plt
import h5py
import numpy as np

In [3]:
class ATLASH5HighLevelDataset(torch.utils.data.Dataset):
    def transform(self,data,transform=True):
        if transform:
            #Calculate some metrics from subsample of total
            vals=[]
            Njets=np.max([1000,data.len()])
            for jet in range(Njets): vals.append(data[jet])
            maxval=np.max(vals)
            minval=np.min(vals)
            return (data-minval)/(maxval-minval)
        else:
            return data

    def __init__(self, file_path, transform=True, return_pt=False):
        super(ATLASH5HighLevelDataset, self).__init__()
        h5_file = h5py.File(file_path , 'r')
        self.data=torch.tensor([])
    
        self.C2=self.transform(h5_file['fjet_C2'],transform=transform)
        self.D2=self.transform(h5_file['fjet_D2'],transform=transform)
        self.ECF1=self.transform(h5_file['fjet_ECF1'],transform=transform)
        self.ECF2=self.transform(h5_file['fjet_ECF2'],transform=transform)
        self.ECF3=self.transform(h5_file['fjet_ECF3'],transform=transform)
        self.L2=self.transform(h5_file['fjet_L2'],transform=transform)
        self.L3=self.transform(h5_file['fjet_L3'],transform=transform)
        self.Qw=self.transform(h5_file['fjet_Qw'],transform=transform)
        self.Split12=self.transform(h5_file['fjet_Split12'],transform=transform)
        self.Split23=self.transform(h5_file['fjet_Split23'],transform=transform)
        self.Tau1_wta=self.transform(h5_file['fjet_Tau1_wta'],transform=transform)
        self.Tau2_wta=self.transform(h5_file['fjet_Tau2_wta'],transform=transform)
        self.Tau3_wta=self.transform(h5_file['fjet_Tau3_wta'],transform=transform)
        self.Tau4_wta=self.transform(h5_file['fjet_Tau4_wta'],transform=transform)
        self.ThrustMaj=self.transform(h5_file['fjet_ThrustMaj'],transform=transform)
        self.m=self.transform(h5_file['fjet_m'],transform=transform)
    
        self.pt = h5_file['fjet_pt']
        self.labels = h5_file['labels']
        if "training_weights" in h5_file:
            self.hasWeights=True
            self.weights = h5_file['training_weights']
        else:
            self.hasWeights=False
    
        self.return_pt = return_pt
    
    def __getitem__(self, index):
        self.data=torch.tensor([self.D2[index],self.C2[index],self.ECF1[index],self.ECF2[index],self.ECF3[index],self.L2[index],self.L3[index],self.Qw[index],self.Split12[index],self.Split23[index],self.Tau1_wta[index],self.Tau2_wta[index],self.Tau3_wta[index],self.Tau4_wta[index],self.ThrustMaj[index],self.m[index]])
    
        if self.return_pt:
            self.data=self.data,torch.tensor([self.pt[index]])
    
        if self.hasWeights:
            return self.data,torch.tensor(self.labels[index],dtype=torch.int64),torch.tensor(self.weights[index])
        else:
            return self.data,torch.tensor(self.labels[index],dtype=torch.int64),torch.tensor(1)
    
    def __len__(self):
        return len(self.labels)

In [4]:
def get_ATLAS_inputs(directory,network,batch_size=2**8,return_pt=False,transform=True):
    
    #Get list of input files
    training_list=[]
    testing_list=[]
    for file in sorted(os.listdir(directory)):
        if not ".h5" in file: continue
        if "train" in file:
            print("Using file %s for training"%(directory+"/"+file))
            training_list.append(directory+"/"+file)
        elif "test" in file:
            print("Using file %s for testing"%(directory+"/"+file))
            testing_list.append(directory+"/"+file)
    
    #concat them
    training_DSlist=[]
    testing_DSlist=[]
    if network!="HL":
        for file in training_list: training_DSlist.append(ATLASH5LowLevelDataset(file,return_pt=return_pt,transform=transform))
        for file in testing_list: testing_DSlist.append(ATLASH5LowLevelDataset(file,return_pt=return_pt,transform=transform))
    else:
        for file in training_list: training_DSlist.append(ATLASH5HighLevelDataset(file,return_pt=return_pt,transform=transform))
        for file in testing_list: testing_DSlist.append(ATLASH5HighLevelDataset(file,return_pt=return_pt,transform=transform))
    training_data = ConcatDataset(training_DSlist)
    testing_data = ConcatDataset(testing_DSlist)
    
    #load them into torch dataloaders 
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=0)
    test_dataloader = DataLoader(testing_data, batch_size=batch_size, shuffle=True)
    
    #return
    return train_dataloader, test_dataloader

In [5]:
class DNNetwork(nn.Module):
    def __init__(self,Ninputs,useConstituents=False):
        super().__init__()
        self.useConstituents=useConstituents
    
        if self.useConstituents: self.flatten = nn.Flatten()
        #self.norm=nn.BatchNorm1d(Ninputs)
        self.fc1= nn.Linear(Ninputs, 512)
        self.act1=nn.ReLU()
        self.fc2= nn.Linear(512, 512)
        self.act2=nn.ReLU()
        self.fc3= nn.Linear(512, 512)
        self.act3=nn.ReLU()
        self.fc4= nn.Linear(512, 2)
    
    def forward(self, x):
        if self.useConstituents: x = self.flatten(x)
        #x=self.norm(x)
        f1=self.act1(self.fc1(x))
        f2=self.act2(self.fc2(f1))
        f3=self.act3(self.fc3(f2))
        logits = self.fc4(f3)
        return logits

In [6]:
train_dataloader,test_dataloader=get_ATLAS_inputs("./","HL",batch_size=2**8,return_pt=True)

Using file .//test_nominal_000.h5 for testing
Using file .//train_nominal_000.h5 for training


In [8]:
Ndim = len(train_dataloader.dataset[0][0][0])
device = "cpu"
model = DNNetwork(Ndim).to(device)
model.load_state_dict(torch.load("model.pth"))
print(model)

<All keys matched successfully>

In [11]:
print(train_dataloader.dataset[0])

((tensor([6.3456e-02, 1.8466e-01, 2.7753e-01, 1.8973e-02, 2.9963e-04, 7.8222e-01,
        5.9016e-01, 6.0296e-02, 2.3117e-02, 5.0525e-02, 9.4161e-02, 1.1921e-01,
        7.9511e-02, 8.6709e-02, 2.8462e-01, 4.4461e-02]), tensor([1438239.7500])), tensor(1), tensor(1.))


In [12]:
#Load information into numpy arrays
ys=[]
preds_raw=[]
weights=[]
pts=[]
with torch.no_grad():
    for (X, pt), y, w in test_dataloader:
        #Get prediction
        pred=model(X)
        pred=torch.sigmoid(pred)

        #append info
        preds_raw.append(pred.numpy())
        ys.append(y.numpy())
        pts.append(pt.numpy())
        weights.append(w.numpy())

In [13]:
#Store things in useful per-jet format
preds_raw=np.array(preds_raw)
preds_raw=preds_raw.reshape(-1,preds_raw.shape[-1])
ys=np.array(ys).flatten()
pts=np.array(pts).flatten()
weights=np.array(weights).flatten()

/Users/les/venv/CometaTutorial/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/Users/les/venv/CometaTutorial/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.
/Users/les/venv/CometaTutorial/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is dep

In [15]:
#make confusion matrix
confusion=metrics.confusion_matrix(ys, np.argmax(preds_raw,axis=1))
print(confusion)

ModuleNotFoundError: No module named 'sklearn'

In [16]:
#Decision rule
preds=np.log(preds_raw[:,0])/np.log(preds_raw[:,1])

TypeError: loop of ufunc does not support argument 0 of type numpy.ndarray which has no callable log method

In [17]:
#make ROC curve
fpr, tpr, thresholds = metrics.roc_curve(ys, preds, sample_weight=weights)
fig, ax = plt.subplots()
ax.plot(tpr,1/fpr)
ax.set_yscale('log')
ax.set_xlim([0.3, 1.0])
fig.savefig(f"ROC_{outname}.pdf") 
plt.close(fig)

NameError: name 'metrics' is not defined

In [18]:
#Find 50% working point
wp50=thresholds[np.argmax(tpr>0.5)]

NameError: name 'thresholds' is not defined

In [19]:
# make rejection plot at working point
Nbins=10
bins=np.linspace(0,3e6,Nbins+1,endpoint=True)
N_accepted=np.zeros(Nbins)
N_all=np.zeros(Nbins)

bkg_indices=np.argwhere(ys==0)
bkg_pts=pts[bkg_indices]
bkg_preds=preds[bkg_indices]
for ii in range(len(bkg_indices)):
    binx=int(np.floor(bkg_pts[ii]/(bins[1])))
    if binx>len(N_all)-1: continue

    N_all[binx]+=1
    if bkg_preds[ii]>wp50: N_accepted[binx]+=1

fig, ax = plt.subplots()
ax.errorbar((bins[1:]+bins[:-1])/2, np.nan_to_num(N_all/N_accepted), xerr=(bins[1:]-bins[:-1])/2,fmt='o')
fig.savefig(f"rejection_pt_{outname}.pdf")
plt.close(fig)

/Users/les/venv/CometaTutorial/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  import sys


NameError: name 'preds' is not defined